In [3]:
import warnings
warnings.filterwarnings('ignore')
import os
import django
import pandas as pd
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'fxfutbol.settings')
django.setup()
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from analysis.models import MatchInfo
from analysis.models import MatchStats



class MatchinfoData:
    def __init__(self):
        pass
        
    def fikturedata(self):
        
        fixture_data = MatchStats.objects.all().values()
        df = pd.DataFrame(list(fixture_data))
        stats_df = pd.json_normalize(df['stats'])
        print(len(stats_df))

        return stats_df
        
    def Processdata(self):
        
        df = self.fikturedata()
        
        df["match_info.data.event.roundInfo.name"]=0
        df["match_info.data.event.roundInfo.slug"]=0
        mycols = [
                    "match_id","match_info.data.event.tournament.name","match_info.data.event.tournament.category.name",
                    "match_info.data.event.tournament.category.id","match_info.data.event.tournament.category.country.alpha3",
                    "match_info.data.event.tournament.uniqueTournament.id","match_info.data.event.tournament.isGroup",
                    "match_info.data.event.tournament.id","match_info.data.event.season.name","match_info.data.event.season.year",
                    "match_info.data.event.season.id","match_info.data.event.roundInfo.round","match_info.data.event.customId",
                    "match_info.data.event.homeTeam.name","match_info.data.event.homeTeam.userCount","match_info.data.event.homeTeam.manager.name",
                    "match_info.data.event.homeTeam.id","match_info.data.event.awayTeam.name","match_info.data.event.awayTeam.userCount",
                    "match_info.data.event.awayTeam.manager.name","match_info.data.event.awayTeam.id","match_info.data.event.homeScore.period1",
                    "match_info.data.event.homeScore.period2","match_info.data.event.homeScore.normaltime","match_info.data.event.awayScore.period1",
                    "match_info.data.event.awayScore.period2","match_info.data.event.awayScore.normaltime","match_info.data.event.hasGlobalHighlights",
                    "match_info.data.event.hasEventPlayerStatistics","match_info.data.event.hasEventPlayerHeatMap","match_info.data.event.startTimestamp",
                    "match_info.data.event.slug","match_info.data.event.roundInfo.name","match_info.data.event.roundInfo.slug"
                   ]
        
        matchdata = df[mycols]
        
        mynewcols = ["match_id", "tournament_name", "tournament_category_name", "tournament_category_id", 
                     "tournament_category_country_code", "unique_tournament_id", "tournament_is_group",
                     "tournament_id", "season_name", "season_year", "season_id", "round_info", "custom_id", 
                     "home_team_name", "home_team_user_count", "home_team_manager_name", "home_team_id", 
                     "away_team_name", "away_team_user_count", "away_team_manager_name", "away_team_id", 
                     "home_score_period1", "home_score_period2", "home_score_normaltime", "away_score_period1", 
                     "away_score_period2", "away_score_normaltime", "has_global_highlights", "has_event_player_statistics",
                     "has_event_player_heat_map", "start_timestamp", "event_slug", "round_info_name", "round_info_slug"]
                
        matchdata.columns = mynewcols    
        matchdata['start_timestamp'] = pd.to_datetime(matchdata['start_timestamp'], unit='s', errors='coerce')
        matchdata = matchdata.fillna(0)
               
        return matchdata

    def add_to_database(self):
        
        matchdata = self.Processdata()
        MatchInfo.objects.filter(match_id__in=matchdata['match_id'].tolist()).delete()
        
        match_data = [
            MatchInfo(
                match_id=row['match_id'],
                tournament_name=row.get('tournament_name', None),
                tournament_category_name=row.get('tournament_category_name', None),
                tournament_category_id=row.get('tournament_category_id', None),
                tournament_category_country_code=row.get('tournament_category_country_code', None),
                unique_tournament_id=row.get('unique_tournament_id', None),
                tournament_is_group=row.get('tournament_is_group', None),
                tournament_id=row.get('tournament_id', None),
                season_name=row.get('season_name', None),
                season_year=row.get('season_year', None),
                season_id=row.get('season_id', None),
                round_info=row.get('round_info', None),
                custom_id=row.get('custom_id', None),
                home_team_name=row.get('home_team_name', None),
                home_team_user_count=row.get('home_team_user_count', None),
                home_team_manager_name=row.get('home_team_manager_name', None),
                home_team_id=row.get('home_team_id', None),
                away_team_name=row.get('away_team_name', None),
                away_team_user_count=row.get('away_team_user_count', None),
                away_team_manager_name=row.get('away_team_manager_name', None),
                away_team_id=row.get('away_team_id', None),
                home_score_period1=row.get('home_score_period1', None),
                home_score_period2=row.get('home_score_period2', None),
                home_score_normaltime=row.get('home_score_normaltime', None),
                away_score_period1=row.get('away_score_period1', None),
                away_score_period2=row.get('away_score_period2', None),
                away_score_normaltime=row.get('away_score_normaltime', None),
                has_global_highlights=row.get('has_global_highlights', None),
                has_event_player_statistics=row.get('has_event_player_statistics', None),
                has_event_player_heat_map=row.get('has_event_player_heat_map', None),
                start_timestamp=row.get('start_timestamp', None),
                event_slug=row.get('event_slug', None),
                round_info_name=row.get('round_info_name', None),
                round_info_slug=row.get('round_info_slug', None),
            )
            for _, row in matchdata.iterrows()
        ]
        
        MatchInfo.objects.bulk_create(match_data)
        print("Mevcut veriler silindi ve yeni veriler eklendi.")
  

mdata =MatchinfoData()
matchdata = mdata.add_to_database()
matchdata

356
Mevcut veriler silindi ve yeni veriler eklendi.
